In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#datasets path

#AD100

TRAIN_PATH_AD = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/AD100/train.csv"
TEST_PATH_AD = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/AD100/verification-subject-disjoint.csv"
IMAGE_PATH_AD = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/AD100/images/"
IMAGE_PATH_AD_AUG = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/AD100/images_aug/"

#LG4000

TRAIN_PATH_LG = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/LG4000/train.csv"
TEST_PATH_LG = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/LG4000/verification-subject-disjoint.csv"
IMAGE_PATH_LG = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/LG4000/images/"

In [ ]:
#Classifier Functions

contacts_classifier = {
    "No": [0, 0, 1],
    "Yes": [0, 1, 0],
    "Cosmetic" : [1, 0, 0]
}

eye_classifier = {
    "Right": [0, 1],
    "Left": [1, 0]
}

In [ ]:
import pandas as pd

df = pd.read_csv(TRAIN_PATH_LG)
df.head(5)

,Image,Subject,Eye,Gender,Race,Contacts,PupilX,PupilY,PupilR,IrisX,IrisY,IrisR,Fold
0,04261d1016,nd1S04261,Left,Male,White,Yes,320,234,35,323,234,124,6
1,04261d1033,nd1S04261,Right,Male,White,Yes,327,248,39,321,246,122,3
2,04261d1036,nd1S04261,Right,Male,White,Yes,324,236,40,318,233,132,3
3,04261d1142,nd1S04261,Left,Male,White,Yes,323,234,37,331,235,130,6
4,04261d1319,nd1S04261,Left,Male,White,Yes,316,235,39,322,236,134,6


In [ ]:
#Reading the data

def list_classes(filepath, name_class):
  df = pd.read_csv(filepath)

  class_list = list(df[name_class])

  return class_list

eye_classifiers = list_classes(TRAIN_PATH_LG, "Eye")
contacts_classifiers = list_classes(TRAIN_PATH_LG, "Contacts")
print(eye_classifiers)
print(contacts_classifiers)

['Left', 'Right', 'Right', 'Left', 'Left', 'Left', 'Right', 'Left', 'Left', 'Right', 'Left', 'Right', 'Right', 'Left', 'Right', 'Left', 'Right', 'Right', 'Left', 'Right', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Right', 'Right', 'Left', 'Left', 'Right', 'Right', 'Right', 'Right', 'Left', 'Left', 'Left', 'Right', 'Right', 'Right', 'Left', 'Left', 'Right', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Right', 'Right', 'Right', 'Right', 'Left', 'Right', 'Left', 'Right', 'Left', 'Right', 'Left', 'Right', 'Left', 'Left', 'Left', 'Left', 'Right', 'Right', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left',

In [ ]:
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
import os
import numpy as np


def make_data(filepath, imagepath = IMAGE_PATH_LG, augmentation=False, dict_eye = eye_classifier, dict_contacts = contacts_classifier):
  images_names = os.listdir(imagepath)

  eye_classification = list_classes(filepath, "Eye")
  contacts_classification = list_classes(filepath, "Contacts")

  images = []
  eye_label, contacts_label = [], []

  index = -1
  for item in images_names:
    index+=1
    if(index == len(eye_classification)):
      break
    image = load_img(imagepath + item, target_size=(224, 224, 3, 1))
    image = img_to_array(image) / 255.0


    eye_list = dict_eye[eye_classification[index]]
    contact_list = dict_contacts[contacts_classification[index]]

    images.append(image)
    eye_label.append(eye_list)
    contacts_label.append(contact_list)

    if augmentation:
      datagen = ImageDataGenerator(
          rotation_range=10,
          zoom_range=0.1,
      )

      i = 0
      for batch in datagen.flow(image, batch_size=8, save_to_dir=IMAGE_PATH_AD_AUG, save_prefix="aug_image", save_format="tiff"):
        images.append(batch)
        eye_label.append(eye_list)
        contacts_label.append(contact_list)

        i+=1
        if i == 8:
          break

  return np.array(images).astype(np.float32), np.array(eye_label).astype(np.int32), np.array(contacts_label).astype(np.int32)

In [ ]:
x_train, y_train_eye, y_train_contact = make_data(TRAIN_PATH_LG)
x_test, y_test_eye, y_test_contact = make_data(TEST_PATH_LG)

print(f"Train: {x_train.shape} \t Eye_Label: {y_train_eye.shape} \t Contact_Label: {y_train_contact.shape}\n")
print(f"Test: {x_test.shape} \t Eye_Label: {y_test_eye.shape} \t Contact_Label: {y_test_contact.shape}\n")

Train: (3000, 224, 224, 3) 	 Eye_Label: (3000, 2) 	 Contact_Label: (3000, 3)

Test: (1200, 224, 224, 3) 	 Eye_Label: (1200, 2) 	 Contact_Label: (1200, 3)



In [ ]:
from keras.applications import VGG16, MobileNet, InceptionV3, ResNet101, InceptionResNetV2, EfficientNetB3

base_model = EfficientNetB3(include_top=False, weights='imagenet',  classifier_activation=None)
for layer in base_model.layers:
  layer.trainable = True


base_model.summary()

Model: "efficientnetb3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, None, None, 3)        0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, None, None, 3)        7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, None, None, 3)        0         ['normalization[0

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, LeakyReLU


def generate_model(base_model):

  input_layer = Input(shape=(224, 224, 3))

  base_model = base_model (input_layer)

  batch_norm = BatchNormalization() (base_model)
  global_pooling = GlobalAveragePooling2D() (batch_norm)

  dropout1 = Dropout(0.1)(global_pooling)
  dense1 = Dense(1024, activation=LeakyReLU())(dropout1)

  dropout2 = Dropout(0.1)(dense1)
  dense2 = Dense(512, activation=LeakyReLU())(dropout2)

  final_dropout = Dropout(0.1)(dense2)
  final_dense   = Dense(512, activation=LeakyReLU())(final_dropout)

  #contacts layer
  contacts_layer = Dense(3, activation='softmax', name="contacts_layer")(final_dense)

  #eye layer
  eye_layer = Dense(2, activation='sigmoid', name="eye_layer")(final_dense)

  model = Model(inputs= input_layer, outputs=[eye_layer, contacts_layer])

  return model


model = generate_model(base_model)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 efficientnetb3 (Functional  (None, None, None, 1536)     1078353   ['input_2[0][0]']             
 )                                                        5                                       
                                                                                                  
 batch_normalization (Batch  (None, 7, 7, 1536)           6144      ['efficientnetb3[0][0]']      
 Normalization)                                                                                   
                                                                                              

In [ ]:
from keras.optimizers import RMSprop, Adam, SGD
from keras.optimizers.schedules import ExponentialDecay
from keras.losses import BinaryCrossentropy, CategoricalCrossentropy

learning_rates = [0.001, 0.0001]
epochs = 20
batch_size = 10

decay_factor = (learning_rates[1] / learning_rates[0]) / epochs
decay_steps = int(len(x_train) / batch_size)

learning_rate = ExponentialDecay(
    initial_learning_rate= learning_rates[0],
    decay_steps= decay_steps,
    decay_rate=decay_factor,
    staircase=True
)

optimizer = Adam(learning_rate=learning_rate)


losses = {
    'eye_layer': BinaryCrossentropy(),
    'contacts_layer': CategoricalCrossentropy()
}

model.compile(optimizer= optimizer, loss= losses, metrics= 'accuracy')

In [ ]:
history = model.fit(x_train, [y_train_eye, y_train_contact], epochs= epochs, batch_size=batch_size, validation_split=0.2, shuffle=True)

Epoch 1/20
240/240 [==============================] - 116s 200ms/step - loss: 1.6318 - eye_layer_loss: 0.7118 - contacts_layer_loss: 0.9200 - eye_layer_accuracy: 0.5050 - contacts_layer_accuracy: 0.5217 - val_loss: 1.5823 - val_eye_layer_loss: 0.6931 - val_contacts_layer_loss: 0.8892 - val_eye_layer_accuracy: 0.4783 - val_contacts_layer_accuracy: 0.9967
Epoch 2/20
240/240 [==============================] - 43s 181ms/step - loss: 1.4758 - eye_layer_loss: 0.6965 - contacts_layer_loss: 0.7793 - eye_layer_accuracy: 0.5088 - contacts_layer_accuracy: 0.6071 - val_loss: 4.3386 - val_eye_layer_loss: 0.6943 - val_contacts_layer_loss: 3.6443 - val_eye_layer_accuracy: 0.5167 - val_contacts_layer_accuracy: 0.0000e+00
Epoch 3/20
240/240 [==============================] - 44s 184ms/step - loss: 1.3968 - eye_layer_loss: 0.6884 - contacts_layer_loss: 0.7084 - eye_layer_accuracy: 0.5346 - contacts_layer_accuracy: 0.6408 - val_loss: 3.1089 - val_eye_layer_loss: 0.6925 - val_contacts_layer_loss: 2.4165 -

In [ ]:
evaluation = model.evaluate(x_test, [y_test_eye, y_test_contact], verbose=1)

38/38 [==============================] - 11s 158ms/step - loss: 2.7967 - eye_layer_loss: 0.7041 - contacts_layer_loss: 2.0926 - eye_layer_accuracy: 0.5092 - contacts_layer_accuracy: 0.3408


In [ ]:
print("Eye's Prediction:     ",evaluation[3] * 100,"%")
print("Contact's Prediction: ",evaluation[4] * 100,"%")

Eye's Prediction:      50.91666579246521 %
Contact's Prediction:  34.08333361148834 %
